In [3]:
def generate_fractal(a, b, red, green, blue, zoom_level=1, max_iter = 250,
                   center_point = (0,0), job = None, directory = None,
                   image_size = (1920, 1080), image_show = False,
                   image_save = True, color_fixing = False,
                   x_max=2.3, aspect_ratio = 16/9, verbose = True):
    ''' 
        Generate Fractals. 
        
        The equation z_next = z**2 + c was iterated max_iter times for each pixel in the image. 
        Now a different function is used.
        z is a complex number that starts out as the coordinates of the pixel on the complex plane (i.e. x + yi). 
        c is a constant (also a complex number) equal to a + bi.
        
        Params:
            a - required: Real component of a + bi
            b - required: Imaginary component of a + bi
            red, green, blue - required: Values that range from 0 - 10. 
                Higher number means more of that color is in the image. 
                For example, if red, green, blue = 10, 0, 10: image will be purple.
            
            max_iter - optional: Number of iterations for the equation z_next = z**2 + c. 
                Generally won't need this to be larger than 250, although there are cases where max_iter = 1000+ is necessary. 
            job - optional: Useful for computing multiple images. Keeps images in the order in which they were generated.
            directory - optional: Name of the folder to save the image to. 
                Note: Does not ensure that the folder exists. Must exist already.
            image_size - optional: Tuple describing the resolution of the image. 
                Default is (1920, 1080), standard resolution.
            image_show - optional: Display the image using an image viewer immediately after it is generated. Does not require saving.
            image_save - optional: Save the image if true.
            color_fixing - optional: Adjusts colors for how human eye percieve colors. 
                Images with this option enabled often look better.
            x_max - optional: Distance from 0 on the real number axis. 
                Generally, julia sets are defined between -2 and 2, so x_max ought to be slightly larger than 2.
            aspect_ratio - optional: Possible values can be 16/9, 9/16, 11/17, 17/11, etc. 
            
        Returns:
            Saves a julia set image in the current directory. (if image_save = True)
            
        Required dependencies:
            PIL
            math
            datetime
            random
    
    '''
    from PIL import Image
    import math
    from datetime import datetime
    from random import randint
    import colorsys
    
    start_time = datetime.now()
    
    if image_size[0]/image_size[1] != aspect_ratio:
        print('Warning: resolution does not match aspect ratio. Resolution: ' + str(image_size))
    
# Find the boundaries of the complex plane in which the fractal will be generated based on the aspect ratio.
    if aspect_ratio > 1:
        y_max = x_max / aspect_ratio
    else:
        y_max = x_max * aspect_ratio
    x_min = -x_max
    y_min = -y_max
    
    x_max = center_point[0] + x_max * 1/zoom_level
    y_max = center_point[1] + y_max * 1/zoom_level
    x_min = center_point[0] + x_min * 1/zoom_level
    y_min = center_point[1] + y_min * 1/zoom_level
#     x_max = center_point[0] + zoom_level
#     x_min = -x_max
#     y_max = x_max/aspect_ratio
#     y_min = -y_max
        
# Initialize a black image, load in the pixels of the image to an array 'pixel'
    image = Image.new('RGB', image_size, 'black')
    pixel = image.load()
    
# Set the scaling factor - the mandelbrot set is defined between -2 and 2, not the pixel size of the image
    x_size = (x_max - x_min)/image.size[0]
    y_size = (y_max - y_min)/image.size[1]
    
    def hsv2rgb(h,s,v):
        return tuple(round(i * 255) for i in colorsys.hsv_to_rgb(h,s,v))
    
# For each pixel in the image, iterate z_next = z^2 + c, 
#   where z is imaginary and c is the (scaled down) value of that specific pixel
    this_time = datetime.now()
    red_perm, green_perm, blue_perm = red, green, blue
    for x in range(image.size[0]):
# Print the progress bar
        if verbose: print_progress_bar(x+1, image.size[0], 'Percentage complete:', 'Finished.')
        for y in range(image.size[1]):
            red, green, blue = red_perm, green_perm, blue_perm
            l = 0 # Initialize the while loop counter
            z = complex((x_min + x * x_size), (y_min + y * y_size))
            z_curr = z
            c = complex(a, b)
            while pixel[x,y] == (0,0,0) and l < max_iter:
                if abs(z_curr) > 2:
                    # calculate a smoothed color value, between 0 and 1
                    nsmooth = (l + 1 - math.log10(math.log10(abs(z_curr)))/math.log10(2))/max_iter
                    if nsmooth == 1:
                        print("nsmooth is 1:", nsmooth, x, y)
                    elif nsmooth > 1:
                        print("nsmooth is greater than 1:", nsmooth, x, y)
                    elif nsmooth < 0:
                        print("nsmooth less than 0:", nsmooth, x, y)
                    elif nsmooth == 0:
                        print("nsmooth equal to 0:", nsmooth, x, y)
                    pixel[x,y] = hsv2rgb(nsmooth, 0.79, 0.59)
#                     if l % 255 == 0:
#                         red = (red + 1) % 10
#                         green = (green + 3) % 10
#                         blue = (blue + 6) % 10
#                     pixel[x,y] = (l*red, l*green, l*blue)
                # math.floor(l*(255/500)) in each (r,g,b) value is pretty cool
                z_curr = z_curr**2 + c
                l += 1
    
    total_time = datetime.now() - start_time # calculate total time in seconds
    total_time = total_time.total_seconds()
    total_time = total_time/60
    
    if verbose: print('fractal created in', round(total_time, 3), 'minutes')
    
# This increases clarity in white regions and blends the transition 
#    between pixels that leave the set at different iterations
    if color_fixing:
        if verbose: print('Fixing colors...')
        red_sum, green_sum, blue_sum = 0, 0, 0
        N = image.size[0] * image.size[1]

        for x in range(image.size[0]):
            for y in range(image.size[1]):
                red_sum += pixel[x, y][0]
                green_sum += pixel[x, y][1]
                blue_sum += pixel[x, y][2]
    
        avg = (red_sum + green_sum + blue_sum)/(N*3)
        if avg == 0:
            if verbose: print("looks like the image is completely black. Will not fix colors.")
        else:
            for x in range(image.size[0]):
                for y in range(image.size[1]):
                    pixel[x, y] = (int((((pixel[x,y][0]/avg) * 0.1)**2)*255), 
                                   int((((pixel[x,y][1]/avg) * 0.1)**2)*255), 
                                   int((((pixel[x,y][2]/avg) * 0.1)**2)*255))
    
# Optional - show the image. Wouldn't want to do this when rendering lots of images
    if image_show:
        image.show()
        
# Name of this image:
    if job != None:
        save_name_list = ['job_' + str(job), '_a_' + str(a) + '_b_' + str(b),
                          '_color_' + str(red_perm) + '_' + str(green_perm) + '_' + str(blue_perm),
                          '.png']
    else:
        save_name_list = ['a_' + str(a) + '_b_' + str(b),
                          '_color_' + str(red_perm) + '_' + str(green_perm) + '_' + str(blue_perm),
                          '.png']
    
# Save the image
    save_name = ''.join(save_name_list)
    if image_save:
        if directory == None:
            if verbose: print('saved as:', save_name)
            image.save(save_name)
        else:
            if verbose: print('saved as:', save_name, 'in the directory:', directory)
            image.save(directory + '/' + save_name)
    else:
        if verbose: print('Will not save the image.')

def print_progress_bar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 25, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

In [122]:
def generate_julia(a, b, red, green, blue, zoom_level=1, max_iter = 250,
                   center_point = (0,0), job = None, directory = None,
                   image_size = (1920, 1080), image_show = False,
                   image_save = True, color_fixing = False,
                   x_max=2.3, aspect_ratio = 16/9, verbose = True):
    ''' 
        Generate Fractals. 
        
        The equation z_next = z**2 + c was iterated max_iter times for each pixel in the image. 
        Now a different function is used.
        z is a complex number that starts out as the coordinates of the pixel on the complex plane (i.e. x + yi). 
        c is a constant (also a complex number) equal to a + bi.
        
        Params:
            a - required: Real component of a + bi
            b - required: Imaginary component of a + bi
            red, green, blue - required: Values that range from 0 - 10. 
                Higher number means more of that color is in the image. 
                For example, if red, green, blue = 10, 0, 10: image will be purple.
            
            max_iter - optional: Number of iterations for the equation z_next = z**2 + c. 
                Generally won't need this to be larger than 250, although there are cases where max_iter = 1000+ is necessary. 
            job - optional: Useful for computing multiple images. Keeps images in the order in which they were generated.
            directory - optional: Name of the folder to save the image to. 
                Note: Does not ensure that the folder exists. Must exist already.
            image_size - optional: Tuple describing the resolution of the image. 
                Default is (1920, 1080), standard resolution.
            image_show - optional: Display the image using an image viewer immediately after it is generated. Does not require saving.
            image_save - optional: Save the image if true.
            color_fixing - optional: Adjusts colors for how human eye percieve colors. 
                Images with this option enabled often look better.
            x_max - optional: Distance from 0 on the real number axis. 
                Generally, julia sets are defined between -2 and 2, so x_max ought to be slightly larger than 2.
            aspect_ratio - optional: Possible values can be 16/9, 9/16, 11/17, 17/11, etc. 
            
        Returns:
            Saves a julia set image in the current directory. (if image_save = True)
            
        Required dependencies:
            PIL
            math
            datetime
            random
    
    '''
    from PIL import Image
    import math
    from datetime import datetime
    from random import randint
    import colorsys
    
    start_time = datetime.now()
    
    if image_size[0]/image_size[1] != aspect_ratio:
        print('Warning: resolution does not match aspect ratio. Resolution: ' + str(image_size))
    
# Find the boundaries of the complex plane in which the fractal will be generated based on the aspect ratio.
    if aspect_ratio > 1:
        y_max = x_max / aspect_ratio
    else:
        y_max = x_max * aspect_ratio
    x_min = -x_max
    y_min = -y_max
    
    x_max = center_point[0] + x_max * 1/zoom_level
    y_max = center_point[1] + y_max * 1/zoom_level
    x_min = center_point[0] + x_min * 1/zoom_level
    y_min = center_point[1] + y_min * 1/zoom_level
#     x_max = center_point[0] + zoom_level
#     x_min = -x_max
#     y_max = x_max/aspect_ratio
#     y_min = -y_max
        
# Initialize a black image, load in the pixels of the image to an array 'pixel'
    image = Image.new('RGB', image_size, 'black')
    pixel = image.load()
    
# Set the scaling factor - the mandelbrot set is defined between -2 and 2, not the pixel size of the image
    x_size = (x_max - x_min)/image.size[0]
    y_size = (y_max - y_min)/image.size[1]
    
    def hsv2rgb(h,s,v):
        return tuple(round(i * 255) for i in colorsys.hsv_to_rgb(h,s,v))
    
# For each pixel in the image, iterate z_next = z^2 + c, 
#   where z is imaginary and c is the (scaled down) value of that specific pixel
    this_time = datetime.now()
    red_perm, green_perm, blue_perm = red, green, blue
    for x in range(image.size[0]):
# Print the progress bar
        if verbose: print_progress_bar(x+1, image.size[0], 'Percentage complete:', 'Finished.')
        for y in range(image.size[1]):
            red, green, blue = red_perm, green_perm, blue_perm
            l = 0 # Initialize the while loop counter
            z = complex((x_min + x * x_size), (y_min + y * y_size))
            z_curr = z
            c = complex(a, b)
            while pixel[x,y] == (0,0,0) and l < max_iter:
                if abs(z_curr) > 2:
                    # calculate a smoothed color value, between 0 and 1
                    nsmooth = (l + 1 - math.log10(math.log10(abs(z_curr)))/math.log10(2))/max_iter
                    if nsmooth == 1:
                        print("nsmooth is 1:", nsmooth, x, y)
                    elif nsmooth > 1:
                        print("nsmooth is greater than 1:", nsmooth, x, y)
                    elif nsmooth < 0:
                        print("nsmooth less than 0:", nsmooth, x, y)
                    elif nsmooth == 0:
                        print("nsmooth equal to 0:", nsmooth, x, y)
                    pixel[x,y] = hsv2rgb(nsmooth, 0.79, 0.59)
#                     if l % 255 == 0:
#                         red = (red + 1) % 10
#                         green = (green + 3) % 10
#                         blue = (blue + 6) % 10
#                     pixel[x,y] = (l*red, l*green, l*blue)
                # math.floor(l*(255/500)) in each (r,g,b) value is pretty cool
                z_curr = z_curr**2 + c
                l += 1
    
    total_time = datetime.now() - start_time # calculate total time in seconds
    total_time = total_time.total_seconds()
    total_time = total_time/60
    
    if verbose: print('fractal created in', round(total_time, 3), 'minutes')
    
# This increases clarity in white regions and blends the transition 
#    between pixels that leave the set at different iterations
    if color_fixing:
        if verbose: print('Fixing colors...')
        red_sum, green_sum, blue_sum = 0, 0, 0
        N = image.size[0] * image.size[1]

        for x in range(image.size[0]):
            for y in range(image.size[1]):
                red_sum += pixel[x, y][0]
                green_sum += pixel[x, y][1]
                blue_sum += pixel[x, y][2]
    
        avg = (red_sum + green_sum + blue_sum)/(N*3)
        if avg == 0:
            if verbose: print("looks like the image is completely black. Will not fix colors.")
        else:
            for x in range(image.size[0]):
                for y in range(image.size[1]):
                    pixel[x, y] = (int((((pixel[x,y][0]/avg) * 0.1)**2)*255), 
                                   int((((pixel[x,y][1]/avg) * 0.1)**2)*255), 
                                   int((((pixel[x,y][2]/avg) * 0.1)**2)*255))
    
# Optional - show the image. Wouldn't want to do this when rendering lots of images
    if image_show:
        image.show()
        
# Name of this image:
    if job != None:
        save_name_list = ['job_' + str(job), '_a_' + str(a) + '_b_' + str(b),
                          '_color_' + str(red_perm) + '_' + str(green_perm) + '_' + str(blue_perm),
                          '.png']
    else:
        save_name_list = ['a_' + str(a) + '_b_' + str(b),
                          '_color_' + str(red_perm) + '_' + str(green_perm) + '_' + str(blue_perm),
                          '.png']
    
# Save the image
    save_name = ''.join(save_name_list)
    if image_save:
        if directory == None:
            if verbose: print('saved as:', save_name)
            image.save(save_name)
        else:
            if verbose: print('saved as:', save_name, 'in the directory:', directory)
            image.save(directory + '/' + save_name)
    else:
        if verbose: print('Will not save the image.')

def print_progress_bar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 25, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

In [119]:
generate_julia(0.359, 0.348, 3, 4, 7, image_size = (960, 540), image_save = False, verbose = False)

nsmooth is greater than 1: 1.0005384284166268 395 131
nsmooth is greater than 1: 1.0024622957880687 455 177
nsmooth is greater than 1: 1.0002183926940975 459 161
nsmooth is greater than 1: 1.0009150826093625 469 300
nsmooth is greater than 1: 1.0009150826093625 491 240
nsmooth is greater than 1: 1.0002183926940975 501 379
nsmooth is greater than 1: 1.0024622957876925 505 363
nsmooth is greater than 1: 1.0005384284166268 565 409


In [120]:
def generate_mandelbrot_zoom(red, green, blue, zoom_level=1, max_iter = 250,
                   center_point = (0,0), job = None, directory = None,
                   image_size = (1920, 1080), image_show = False,
                   image_save = True, color_fixing = False,
                   x_max=2.3, aspect_ratio = 16/9, verbose = True):
    ''' 
        Generate Fractals. 
        
        The equation z_next = z**2 + c was iterated max_iter times for each pixel in the image. 
        Now a different function is used.
        z is a complex number that starts out as the coordinates of the pixel on the complex plane (i.e. x + yi). 
        c is a constant (also a complex number) equal to a + bi.
        
        Params:
            a - required: Real component of a + bi
            b - required: Imaginary component of a + bi
            red, green, blue - required: Values that range from 0 - 10. 
                Higher number means more of that color is in the image. 
                For example, if red, green, blue = 10, 0, 10: image will be purple.
            
            max_iter - optional: Number of iterations for the equation z_next = z**2 + c. 
                Generally won't need this to be larger than 250, although there are cases where max_iter = 1000+ is necessary. 
            job - optional: Useful for computing multiple images. Keeps images in the order in which they were generated.
            directory - optional: Name of the folder to save the image to. 
                Note: Does not ensure that the folder exists. Must exist already.
            image_size - optional: Tuple describing the resolution of the image. 
                Default is (1920, 1080), standard resolution.
            image_show - optional: Display the image using an image viewer immediately after it is generated. Does not require saving.
            image_save - optional: Save the image if true.
            color_fixing - optional: Adjusts colors for how human eye percieve colors. 
                Images with this option enabled often look better.
            x_max - optional: Distance from 0 on the real number axis. 
                Generally, julia sets are defined between -2 and 2, so x_max ought to be slightly larger than 2.
            aspect_ratio - optional: Possible values can be 16/9, 9/16, 11/17, 17/11, etc. 
            
        Returns:
            Saves a julia set image in the current directory. (if image_save = True)
            
        Required dependencies:
            PIL
            math
            datetime
            random
    
    '''
    from PIL import Image
    import math
    from datetime import datetime
    from random import randint
    import colorsys
    
    start_time = datetime.now()
    
    if image_size[0]/image_size[1] != aspect_ratio:
        print('Warning: resolution does not match aspect ratio. Resolution: ' + str(image_size))
    
# Find the boundaries of the complex plane in which the fractal will be generated based on the aspect ratio.
    if aspect_ratio > 1:
        y_max = x_max / aspect_ratio
    else:
        y_max = x_max * aspect_ratio
    x_min = -x_max
    y_min = -y_max
    
    x_max = center_point[0] + x_max * 1/zoom_level
    y_max = center_point[1] + y_max * 1/zoom_level
    x_min = center_point[0] + x_min * 1/zoom_level
    y_min = center_point[1] + y_min * 1/zoom_level
#     x_max = center_point[0] + zoom_level
#     x_min = -x_max
#     y_max = x_max/aspect_ratio
#     y_min = -y_max
        
# Initialize a black image, load in the pixels of the image to an array 'pixel'
    image = Image.new('RGB', image_size, 'black')
    pixel = image.load()
    
# Set the scaling factor - the mandelbrot set is defined between -2 and 2, not the pixel size of the image
    x_size = (x_max - x_min)/image.size[0]
    y_size = (y_max - y_min)/image.size[1]
    
    def hsv2rgb(h,s,v):
        return tuple(round(i * 255) for i in colorsys.hsv_to_rgb(h,s,v))
    
# For each pixel in the image, iterate z_next = z^2 + c, 
#   where z is imaginary and c is the (scaled down) value of that specific pixel
    this_time = datetime.now()
    red_perm, green_perm, blue_perm = red, green, blue
    for x in range(image.size[0]):
# Print the progress bar
        if verbose: print_progress_bar(x+1, image.size[0], 'Percentage complete:', 'Finished.')
        for y in range(image.size[1]):
            red, green, blue = red_perm, green_perm, blue_perm
            l = 0 # Initialize the while loop counter
            z = complex((x_min + x * x_size), (y_min + y * y_size))
            z_curr = z
            while pixel[x,y] == (0,0,0) and l < max_iter:
                if abs(z_curr) > 2:
                    # calculate a smoothed color value, between 0 and 1
                    nsmooth = (l + 1 - math.log10(math.log10(abs(z_curr)))/math.log10(2))/max_iter
                    if nsmooth == 1:
                        print("nsmooth is 1:", nsmooth, x, y)
                    elif nsmooth > 1:
                        print("nsmooth is greater than 1:", nsmooth, x, y)
                    elif nsmooth < 0:
                        print("nsmooth less than 0:", nsmooth, x, y)
                    elif nsmooth == 0:
                        print("nsmooth equal to 0:", nsmooth, x, y)
                    pixel[x,y] = hsv2rgb(nsmooth, 0.79, 0.59)
#                     if l % 255 == 0:
#                         red = (red + 1) % 10
#                         green = (green + 3) % 10
#                         blue = (blue + 6) % 10
#                     pixel[x,y] = (l*red, l*green, l*blue)
                # math.floor(l*(255/500)) in each (r,g,b) value is pretty cool
                z_curr = z_curr**2 + z
                l += 1
    
    total_time = datetime.now() - start_time # calculate total time in seconds
    total_time = total_time.total_seconds()
    total_time = total_time/60
    
    if verbose: print('julia set created in', round(total_time, 4), 'minutes')
    
# This increases clarity in white regions and blends the transition 
#    between pixels that leave the set at different iterations
    if color_fixing:
        if verbose: print('Fixing colors...')
        red_sum, green_sum, blue_sum = 0, 0, 0
        N = image.size[0] * image.size[1]

        for x in range(image.size[0]):
            for y in range(image.size[1]):
                red_sum += pixel[x, y][0]
                green_sum += pixel[x, y][1]
                blue_sum += pixel[x, y][2]
    
        avg = (red_sum + green_sum + blue_sum)/(N*3)
        if avg == 0:
            if verbose: print("looks like the image is completely black. Will not fix colors.")
        else:
            for x in range(image.size[0]):
                for y in range(image.size[1]):
                    pixel[x, y] = (int((((pixel[x,y][0]/avg) * 0.1)**2)*255), 
                                   int((((pixel[x,y][1]/avg) * 0.1)**2)*255), 
                                   int((((pixel[x,y][2]/avg) * 0.1)**2)*255))
    
# Optional - show the image. Wouldn't want to do this when rendering lots of images
    if image_show:
        image.show()
        
# Name of this image:
    if job != None:
        save_name_list = ['job_' + str(job),
                          '_color_' + str(red_perm) + '_' + str(green_perm) + '_' + str(blue_perm),
                          '.png']
    else:
        save_name_list = ['color_' + str(red_perm) + '_' + str(green_perm) + '_' + str(blue_perm),
                          '.png']
    
# Save the image
    save_name = ''.join(save_name_list)
    if image_save:
        if directory == None:
            if verbose: print('saved as:', save_name)
            image.save(save_name)
        else:
            if verbose: print('saved as:', save_name, 'in the directory:', directory)
            image.save(directory + '/' + save_name)
    else:
        if verbose: print('Will not save the image.')

def print_progress_bar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 25, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

In [121]:
generate_mandelbrot_zoom(3, 5, 8,
                         zoom_level=1, #center_point=(0.391, -0.334), directory = "fourth_zoom_gif", image_size=(960, 540), 
                         image_save=False, verbose=False)

nsmooth is greater than 1: 1.0006336088785246 471 458
nsmooth is greater than 1: 1.0006336082960678 471 622
nsmooth is greater than 1: 1.0000269328694955 694 379
nsmooth is greater than 1: 1.0000269328694955 694 701
nsmooth is greater than 1: 1.0007523468755002 715 351
nsmooth is greater than 1: 1.0007523468588233 715 729
nsmooth is greater than 1: 1.001219975073718 780 303
nsmooth is greater than 1: 1.0012199754669395 780 777
nsmooth is greater than 1: 1.0022935001365465 967 267
nsmooth is greater than 1: 1.0022935001365465 967 813


In [74]:
def generate_gif(directory, print_file_names = False):
    import imageio
    from glob import glob
    from natsort import natsorted
    
    images = []
    filenames = glob(directory + '/*.png') # Create a list of file names in the specified directory
    
    filenames = natsorted(filenames, key=lambda y: y.lower()) 
    # Sort the list 'filenames' using the traditional method.
    # Traditional method - isolate the entire first number in the string, then sort by that number. 
    # If this step is not included, files will be sorted like so: 0, 100, 110, 200, 3, 420, etc...
    
    if print_file_names: # For troubleshooting
        for i in filenames:
            print(i)
    
    for filename in filenames: 
        images.append(imageio.imread(filename)) 
    # Append each file to the list that will become the gif
    
    imageio.mimsave(directory + '.gif', images) 
    # Save the gif as the name of the directory that the images were generated from

In [105]:
generate_gif('fourth_zoom_gif')